In [ ]:
!pip install -U langchain langchain-google-genai google-generativeai azure-storage-blob


INFO: pip is looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.0/407.0 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.1/207.1 kB 15.3 MB/s eta 0:00:00


In [ ]:
import os
import getpass

os.environ["GOOGLE_API_KEY"] = getpass.getpass("🔐 Enter your Gemini API Key: ")


🔐 Enter your Gemini API Key: ··········


In [ ]:
from azure.storage.blob import BlobServiceClient
import json

AZURE_CONNECTION_STRING = ""
CONTAINER_NAME = "soap-notes"

blob_service_client = BlobServiceClient.from_connection_string(AZURE_CONNECTION_STRING)
container_client = blob_service_client.get_container_client(CONTAINER_NAME)

def get_patient_note(patient_id):
    blob_name = f"patient_{patient_id}.json"
    blob_client = container_client.get_blob_client(blob_name)
    blob_data = blob_client.download_blob().readall()
    return json.loads(blob_data)


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableSequence

llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro-001", temperature=0.3)

prompt = PromptTemplate.from_template("""
You are a medical assistant. Summarize the following patient SOAP note in 4-5 sentences:

{note}
""")

# LangChain flow
chain: RunnableSequence = prompt | llm | StrOutputParser()


In [ ]:
note_text = json.dumps(note_json, indent=2)
summary = chain.invoke({"note": note_text})

print("Summary:\n", summary)


Summary:
 A 45-year-old female patient presented with a chief complaint of shortness of breath and chest tightness for the past week, accompanied by cough, fatigue, and difficulty sleeping. Her medical history is significant for hypertension, diabetes, and osteoarthritis. Physical examination revealed normal vital signs and unremarkable findings. The primary diagnosis is pulmonary embolism, with differential diagnoses including pneumonia, COPD, asthma, and anaphylaxis. The plan includes arterial blood gas analysis, echocardiogram, CT angiography, anticoagulant therapy, bronchodilators, oxygen therapy, and a follow-up appointment with a pulmonologist in one week. 


In [ ]:
from google.generativeai import GenerativeModel, list_models
from google.generativeai import configure

configure(api_key=os.environ["GOOGLE_API_KEY"])
models = list_models()
for m in models:
    print(m.name, "-", m.supported_generation_methods)


models/chat-bison-001 - ['generateMessage', 'countMessageTokens']
models/text-bison-001 - ['generateText', 'countTextTokens', 'createTunedTextModel']
models/embedding-gecko-001 - ['embedText', 'countTextTokens']
models/gemini-1.0-pro-vision-latest - ['generateContent', 'countTokens']
models/gemini-pro-vision - ['generateContent', 'countTokens']
models/gemini-1.5-pro-latest - ['generateContent', 'countTokens']
models/gemini-1.5-pro-001 - ['generateContent', 'countTokens', 'createCachedContent']
models/gemini-1.5-pro-002 - ['generateContent', 'countTokens', 'createCachedContent']
models/gemini-1.5-pro - ['generateContent', 'countTokens']
models/gemini-1.5-flash-latest - ['generateContent', 'countTokens']
models/gemini-1.5-flash-001 - ['generateContent', 'countTokens', 'createCachedContent']
models/gemini-1.5-flash-001-tuning - ['generateContent', 'countTokens', 'createTunedModel']
models/gemini-1.5-flash - ['generateContent', 'countTokens']
models/gemini-1.5-flash-002 - ['generateContent

In [ ]:
# Install pyngrok if not already
!pip install pyngrok flask

from pyngrok import ngrok
import subprocess
import time

# Open a tunnel to port 5000
public_url = ngrok.connect(5000)
print("🌍 Public URL:", public_url)

# Run the Flask app
process = subprocess.Popen(["python3", "/content/app.py"])

# Let it run in the background
time.sleep(2)


🌍 Public URL: NgrokTunnel: "https://39a0-34-139-76-126.ngrok-free.app" -> "http://localhost:5000"


In [ ]:
from pyngrok import ngrok, conf
import getpass

# Paste your authtoken here when prompted
authtoken = getpass.getpass("🔐 Enter your ngrok authtoken: ")
conf.get_default().auth_token = authtoken


🔐 Enter your ngrok authtoken: ··········


In [ ]:
public_url = ngrok.connect(5000)
print("🌍 Public URL:", public_url)


🌍 Public URL: NgrokTunnel: "https://409e-34-139-76-126.ngrok-free.app" -> "http://localhost:5000"


In [ ]:
!pip install flask langchain langchain-google-genai azure-storage-blob


In [ ]:
import os
os.environ["GOOGLE_API_KEY"] = "your-gemini-api-key"


In [ ]:
!python "/content/app.py"


 * Serving Flask app 'app'
 * Debug mode: on
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 640-689-267
005
^C


In [ ]:
from flask import Flask, request, jsonify
import os
import json
import threading
import getpass
from azure.storage.blob import BlobServiceClient

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableSequence

# 🛡 Prompt for setup inputs
os.environ["GOOGLE_API_KEY"] = getpass.getpass("🔐 Enter your Gemini API Key: ")
AZURE_CONNECTION_STRING = getpass.getpass("🔐 Enter your Azure Blob connection string: ")
CONTAINER_NAME = input("📦 Enter your Azure Blob container name (e.g., 'soap-notes'): ")

# 🌩 Azure Blob Client
blob_service_client = BlobServiceClient.from_connection_string(AZURE_CONNECTION_STRING)
container_client = blob_service_client.get_container_client(CONTAINER_NAME)

# 🤖 LangChain Chain with Gemini
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro-001", temperature=0.3)

prompt = PromptTemplate.from_template("""
You are a medical assistant. Summarize the following patient SOAP note in 4-5 clear, professional sentences for a doctor to review:

{note}
""")

summary_chain: RunnableSequence = prompt | llm | StrOutputParser()

# ⚙️ Flask App Setup
app = Flask(__name__)

# 🧠 Read patient note from Azure
def get_patient_note(patient_id):
    try:
        blob_name = f"patient_{patient_id}.json"
        blob_client = container_client.get_blob_client(blob_name)
        blob_data = blob_client.download_blob().readall()
        return json.loads(blob_data)
    except Exception as e:
        raise FileNotFoundError(f"Error loading patient_{patient_id}.json: {e}")

# 🧠 Generate Summary
def generate_summary(note_json):
    note_text = json.dumps(note_json, indent=2)
    return summary_chain.invoke({"note": note_text})

# 🌐 API Endpoint
@app.route("/summarize", methods=["GET"])
def summarize():
    patient_id = request.args.get("patient_id")
    if not patient_id:
        return jsonify({"error": "Missing patient_id"}), 400

    try:
        note = get_patient_note(patient_id)
        summary = generate_summary(note)
        return jsonify({
            "patient_id": patient_id,
            "summary": summary
        })
    except FileNotFoundError as e:
        return jsonify({"error": str(e)}), 404
    except Exception as e:
        return jsonify({"error": f"Unexpected error: {e}"}), 500

# 🚀 Run Flask App (with optional pyngrok support)
def run_flask_app():
    app.run(port=5000)

thread = threading.Thread(target=run_flask_app)
thread.start()


🔐 Enter your Gemini API Key: ··········
🔐 Enter your Azure Blob connection string: ··········
📦 Enter your Azure Blob container name (e.g., 'soap-notes'): soap-notes
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
import requests

patient_id = input("🧑‍⚕️ Enter the patient ID (e.g., '002'): ")

response = requests.get("http://localhost:5000/summarize", params={"patient_id": patient_id})
print("📋 Summary Response:")
print(response.json())


🧑‍⚕️ Enter the patient ID (e.g., '002'): 004


INFO:werkzeug:127.0.0.1 - - [16/Apr/2025 23:39:25] "GET /summarize?patient_id=004 HTTP/1.1" 200 -


📋 Summary Response:
{'patient_id': '004', 'summary': 'This 55-year-old female patient presents with a chief complaint of chronic cough and shortness of breath, accompanied by wheezing and chest tightness, for several months. Her medical history is significant for hypertension and asthma, with a family history of asthma in her mother. Physical examination reveals wheezing breath sounds bilaterally and tenderness in the right upper quadrant upon deep palpation.  The primary diagnosis is asthma exacerbation with bronchitis, with differentials including COPD, lung cancer, and pneumonia.  The plan is to order spirometry, a chest x-ray, and bloodwork, and to start the patient on bronchodilators, corticosteroids, and long-acting bronchodilators.  A follow-up appointment in one week and a referral to pulmonology will be arranged. '}


In [ ]:
!pip install pyngrok

In [ ]:

from pyngrok import ngrok
ngrok.set_auth_token(getpass.getpass("🔐 Enter your ngrok auth token: "))
public_url = ngrok.connect(5000)
print("🌍 Public Flask URL:", public_url)


🔐 Enter your ngrok auth token: ··········
🌍 Public Flask URL: NgrokTunnel: "https://f538-34-139-76-126.ngrok-free.app" -> "http://localhost:5000"
